[![Binder](../img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignments/assignment-performance-evaluation.ipynb)&emsp;
[![Script](../img/badge-script.svg)](/Teaching//assignments/assignment-performance-evaluation.fsx)&emsp;
[![Notebook](../img/badge-notebook.svg)](/Teaching//assignments/assignment-performance-evaluation.ipynb)
Group Name:

Student Name | Student Number
--- | ---
**1:Bernardo Manarte** |55810 &#32;
**2:Leonor Rodrigues Pereira** |48778 &#32;
**3:Marouan Kamoun** |53833 &#32;
**4:Rodrigo Simões** |53154 &#32;
**5:Vasco Calxito** |53960 &#32;


This is an assignment. You should work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.

For testing



In [36]:
#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
open Xunit
open FsUnit.Xunit
open FsUnitTyped


Error: xunit version 2.* cannot be added because version 2.4.2 was added previously.

For the assignment



In [37]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: Plotly.NET"


Installed Packages FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 FsUnit.Xunit, 5.3.0 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1

In [38]:
#r "nuget: Plotly.NET.Interactive, 3.*"


Error: Plotly.NET.Interactive version 3.* cannot be added because version 3.0.2 was added previously.

In [39]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open MathNet.Numerics.Statistics
open NovaSBE.Finance.Ols


## Load Data

First, make sure that you're referencing the correct files.

Here I'm assuming that you have a class folder with this notebook and a `data` folder inside of it. The folder hierarchy would look like below where you
have the below files and folders accessible:

```code
/class
    notebook.ipynb
    id_and_return_data.csv
    rd_sale.csv
    
```


In [40]:
open NovaSBE.Finance.Portfolio


### Data files

We assume the `id_and_return_data.csv` file and the signal csv file  are in the notebook folder.



In [41]:
let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "rd_sale.csv"
let strategyName = "R&D to sales"


If my paths are correct, then this code should read the first few lines of the files.
If it doesn't show the first few lines, fix the above file paths.



In [42]:
IO.File.ReadLines(IdAndReturnsFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


"id(string),eom(date),source(string),sizeGrp(string),obsMain(bool),exchMain(bool),primarySec(bool),gvkey(string),iid(string),permno(int Option),permco(int Option),excntry(string),curcd(string),fx(string),common(bool),compTpci(string),crspShrcd(int Option),compExchg(string),crsp_exchcd(int Option),adjfct(float Option),shares(float Option),gics(int Option),sic(int Option),naics(int Option),ff49(int Option),ret(float Option),retExc(float Option),prc(float Option),marketEquity(float Option)"
"crsp_86432,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115876,01,86432,16313,USA,USD,1,true,,11,,3,2,5.218,40101010,6020,522110,45,-0.003906,-0.00824925,15.9375,83.161875"
"crsp_85640,2000-01-31T00:00:00.0000000,CRSP,small,1,1,true,002193,01,85640,20300,USA,USD,1,true,,11,,1,1,102.496,35102020,8051,623110,11,-0.157143,-0.161485863,3.6875,377.954"
"crsp_86430,2000-01-31T00:00:00.0000000,CRSP,micro,1,1,true,115946,01,86430,16319,USA,USD,1,true,,11,,3,1,10.764,45103010,7372,511210,36,0.285714,0.28137

In [43]:
IO.File.ReadLines(MySignalFilePath) 
|> Seq.truncate 5
|> Seq.iter (printfn "%A")


Error: System.IO.FileNotFoundException: Could not find file '/Users/rodrigosimoes/Documents/Documents - MacBook Pro de Rodrigo/Nova SBE/2nd semester/Data Analytics for Finance/Assignment 6/Final files/rd_sale.csv'.
File name: '/Users/rodrigosimoes/Documents/Documents - MacBook Pro de Rodrigo/Nova SBE/2nd semester/Data Analytics for Finance/Assignment 6/Final files/rd_sale.csv'
   at Interop.ThrowExceptionForIoErrno(ErrorInfo errorInfo, String path, Boolean isDirError)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, UnixFileMode openPermissions, Int64& fileLength, UnixFileMode& filePermissions, Func`4 createOpenException)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String 

Ok, now assuming those paths were correct the below code will work.
I will put all this prep code in one block so that it is easy to run.



In [44]:
let idAndReturnsCsv = 
    CsvProvider<IdAndReturnsFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList
let mySignalCsv = 
    CsvProvider<MySignalFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList


A list of `Signal` records. The signal type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L178-L181).



In [45]:
let mySignals =
    mySignalCsv
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal }
            Some signalRecord)

// look at a few signals
mySignals[..3]


Error: input.fsx (4,19)-(4,25) typecheck error The type 'Row' does not define the field, constructor or member 'Signal'.
input.fsx (8,42)-(8,44) typecheck error The type 'Row' does not define the field, constructor or member 'Id'.
input.fsx (9,48)-(9,51) typecheck error The type 'Row' does not define the field, constructor or member 'Eom'.
input.fsx (9,62)-(9,65) typecheck error The type 'Row' does not define the field, constructor or member 'Eom'.

A list of Security return records. The `SecurityReturn` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L173-L176)



In [46]:
let myReturns =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

// look at a few returns
myReturns[..3]


index value 0 { SecurityId = Other "crsp_86432"\n Date = 01/01/2000 00:00:00\n Return = -0.003906 } SecurityId Other "crsp_86432" Item crsp_86432 Date 2000-01-01 00:00:00Z Return -0.003906 1 { SecurityId = Other "crsp_85640"\n Date = 01/01/2000 00:00:00\n Return = -0.157143 } SecurityId Other "crsp_85640" Item crsp_85640 Date 2000-01-01 00:00:00Z Return -0.157143 2 { SecurityId = Other "crsp_86430"\n Date = 01/01/2000 00:00:00\n Return = 0.285714 } SecurityId Other "crsp_86430" Item crsp_86430 Date 2000-01-01 00:00:00Z Return 0.285714 3 { SecurityId = Other "crsp_85756"\n Date = 01/01/2000 00:00:00\n Return = 0.190685 } SecurityId Other "crsp_85756" Item crsp_85756 Date 2000-01-01 00:00:00Z Return 0.190685

A list of security market caps. We'll need this for value-weight portfolios. The `WeightVariable` type is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L183-L186).



In [47]:
let myMktCaps =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

// look at a few market caps
myMktCaps[..3]


index value 0 { SecurityId = Other "crsp_86432"\n FormationDate = 01/01/2000 00:00:00\n Value = 83.161875 } SecurityId Other "crsp_86432" Item crsp_86432 FormationDate 2000-01-01 00:00:00Z Value 83.161875 1 { SecurityId = Other "crsp_85640"\n FormationDate = 01/01/2000 00:00:00\n Value = 377.954 } SecurityId Other "crsp_85640" Item crsp_85640 FormationDate 2000-01-01 00:00:00Z Value 377.954 2 { SecurityId = Other "crsp_86430"\n FormationDate = 01/01/2000 00:00:00\n Value = 42.38325 } SecurityId Other "crsp_86430" Item crsp_86430 FormationDate 2000-01-01 00:00:00Z Value 42.38325 3 { SecurityId = Other "crsp_85756"\n FormationDate = 01/01/2000 00:00:00\n Value = 16.2155 } SecurityId Other "crsp_85756" Item crsp_85756 FormationDate 2000-01-01 00:00:00Z Value 16.2155

## Forming our strategy

We're now going to use the `Backtest` code to generate portfolios. It is defined in the `NovaSBE.Finance.Portfolio` module [here](https://github.com/nhirschey/NovaSBE.Finance/blob/6d1398625e5a9279af00bb6e1c1802af3596c3f6/src/NovaSBE.Finance/Portfolio.fs#L199).
The `Backtest` class automates some of the code we did earlier to make portfolio construction simpler.



In [48]:
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)


Error: input.fsx (1,52)-(1,61) typecheck error The value or constructor 'mySignals' is not defined. Maybe you want one of the following:
   mySignalCsv
   MySignalFilePath
   Signal

### Value Weighted Portfolios



In [49]:
let vw = backtest.strategyValueWeighted(myMktCaps)

vw.Portfolios[..3]


Error: input.fsx (1,10)-(1,18) typecheck error The value, namespace, type or module 'backtest' is not defined. Maybe you want one of the following:
   Backtest
   BackTestResult
   Bucket

In [50]:
vw.Returns[..3]


Error: input.fsx (1,1)-(1,3) typecheck error The value, namespace, type or module 'vw' is not defined.

### Long-short portfolios

We get the Fama-French 3-Factor asset pricing model data.



In [51]:
open NovaSBE.Finance

let ff3Lookup = 
    French.getFF3 French.Frequency.Monthly
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month, 1), x)
    |> Map


Isolate some notable portfolios.



In [52]:
type SignalPortfolioObs = 
    { Month: DateTime
      Name: string
      ExcessReturn: float }
let long =
    vw.Returns
    |> List.filter (fun row -> row.Index = 3)
    |> List.map (fun row ->
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Long"
          ExcessReturn = retx })

let short =
    vw.Returns
    |> List.filter (fun row -> row.Index = 1)
    |> List.map (fun row -> 
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Short"
          ExcessReturn = retx })

let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Month = long.Month
          Name = "Long-short"
          ExcessReturn = long.Return - short.Return })


Error: input.fsx (6,5)-(6,7) typecheck error The value, namespace, type or module 'vw' is not defined.
input.fsx (9,24)-(9,30) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Return'.
input.fsx (9,47)-(9,52) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Month'.
input.fsx (10,23)-(10,28) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Month'.
input.fsx (15,5)-(15,7) typecheck error The value, namespace, type or module 'vw' is not defined.
input.fsx (18,24)-(18,30) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Return'.
input.fsx (18,47)-(18,52) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Month'.
input.fsx (19,23)-(19,28) typecheck error The type 'AssignedPortfolio' does not define the field, constructor or member 'Month'.
input.fsx (24,5)-(24,7) typ

## Start of assignment

> **Task:** Calculate the annualized Sharpe ratios of your long, short, and long-short portfolios. Assign them to values `longSharpe`, `shortSharpe`, and `longShortSharpe`, respectively.
> 



In [53]:
let annualizeSharpeRatio (excessReturns: SignalPortfolioObs list) =
    let excessReturnsSeq = excessReturns |> List.toSeq
    let avgExcessReturn = excessReturnsSeq |> Seq.averageBy (fun x -> x.ExcessReturn)
    let stDevExcessReturn = excessReturnsSeq |> Seq.map (fun x -> x.ExcessReturn) |> Seq.stDev
    let sharpeRatio = avgExcessReturn / stDevExcessReturn
    sharpeRatio * sqrt 12.0

let longSharpe = annualizeSharpeRatio long
let shortSharpe = annualizeSharpeRatio short
let longShortSharpe = annualizeSharpeRatio longShort

(longSharpe, shortSharpe, longShortSharpe)


Error: input.fsx (1,42)-(1,60) typecheck error The type 'SignalPortfolioObs' is not defined. Maybe you want one of the following:
   Signal
   AssignedPortfolio
input.fsx (3,71)-(3,85) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (4,67)-(4,81) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (8,39)-(8,43) typecheck error The value or constructor 'long' is not defined. Maybe you want one of the following:
   log
   log2
input.fsx (9,40)-(9,45) typecheck error The value or constructor 'short' is not defined. Maybe you want one of the following:
   sqrt
   Chart
input.fsx (10,44)-(1

Tests



In [54]:
longSharpe |> should (equalWithin 1e-6) 0.2613490337
shortSharpe |> should (equalWithin 1e-6) 0.490118588
longShortSharpe |> should (equalWithin 1e-6) -0.05676492997


Error: input.fsx (1,1)-(1,11) typecheck error The value or constructor 'longSharpe' is not defined.
input.fsx (2,1)-(2,12) typecheck error The value or constructor 'shortSharpe' is not defined.
input.fsx (3,1)-(3,16) typecheck error The value or constructor 'longShortSharpe' is not defined.

> **Task:** Create a `list<RegData>` for your  long-short portfolio. Assign it to a value named `longShortRd`.
> 



In [55]:
type RegData =
    { Month: DateTime
      ExcessReturn: float 
      MktRf: float
      Hml: float
      Smb: float }

// Solution here.
let longShortRd =
    longShort
    |> List.map (fun ls ->
        let ffFactors = ff3Lookup[ls.Month]
        { Month = ls.Month
          ExcessReturn = ls.ExcessReturn
          MktRf = ffFactors.MktRf
          Hml = ffFactors.Hml
          Smb = ffFactors.Smb })

// Display the first few elements of the longShortRd list
longShortRd |> List.take 5


Error: input.fsx (10,5)-(10,14) typecheck error The value or constructor 'longShort' is not defined.

Tests



In [56]:
longShortRd |> shouldHaveLength 252
longShortRd |> should be ofExactType<RegData list>


Error: input.fsx (1,1)-(1,12) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (2,1)-(2,12) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (2,38)-(2,45) typecheck error The type 'RegData' is not defined.

> **Task:** Fit CAPM and Fama-French 3-factor models for your long-short portfolio. Assign them to a values named `capmModel` and `ff3Model`, respectively.
> 



In [57]:
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Statistics

type RegressionResults =
    { Betas: float array
      Residuals: float array
      Mse: float }

let capmInputData = 
    longShortRd
    |> List.map (fun rd -> [|1.0; rd.MktRf|], rd.ExcessReturn)

let ff3InputData = 
    longShortRd
    |> List.map (fun rd -> [|1.0; rd.MktRf; rd.Hml; rd.Smb|], rd.ExcessReturn)

let capmX = capmInputData |> List.map fst |> Array.ofList |> DenseMatrix.ofRowArrays
let capmY = capmInputData |> List.map snd |> Array.ofList |> DenseVector.ofArray

let ff3X = ff3InputData |> List.map fst |> Array.ofList |> DenseMatrix.ofRowArrays
let ff3Y = ff3InputData |> List.map snd |> Array.ofList |> DenseVector.ofArray

let capmBetas = capmX.QR().Solve(capmY)
let capmResiduals = capmY - (capmX * capmBetas)
let capmMse = capmResiduals.PointwisePower(2).Mean()

let ff3Betas = ff3X.QR().Solve(ff3Y)
let ff3Residuals = ff3Y - (ff3X * ff3Betas)
let ff3Mse = ff3Residuals.PointwisePower(2).Mean()

let capmModel = { Betas = capmBetas.ToArray(); Residuals = capmResiduals.ToArray(); Mse = capmMse }
let ff3Model = { Betas = ff3Betas.ToArray(); Residuals = ff3Residuals.ToArray(); Mse = ff3Mse }

(capmModel, ff3Model)


Error: input.fsx (10,5)-(10,16) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (11,35)-(11,43) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (11,47)-(11,62) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (14,5)-(14,16) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (15,35)-(15,43) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be r

Tests.



In [58]:
capmModel |> should be ofExactType<RegressionResults>
ff3Model |> should be ofExactType<RegressionResults>


Error: input.fsx (1,1)-(1,10) typecheck error The value or constructor 'capmModel' is not defined.
input.fsx (2,1)-(2,9) typecheck error The value or constructor 'ff3Model' is not defined.

### CAPM model evaluation.

> **Task:** What is the CAPM alpha for your long-short portfolio? Use code to assign the alpha to the value `capmAlpha`. Is it significantly different from zero? Use code to assign the t-statistic to the value `capmAlphaT`.
> 


In [59]:
open MathNet.Numerics

let capmAlpha = capmModel.Betas.[0]

let capmX = longShortRd |> List.map (fun r -> [|1.0; r.MktRf|]) |> List.toArray |> DenseMatrix.ofRowArrays

let residuals = capmModel.Residuals
let n = float residuals.Length

let XTX = capmX.TransposeThisAndMultiply(capmX)
let XTX_inv = XTX.Inverse()

let sumSquaredResiduals = residuals |> Array.map (fun x -> x * x) |> Array.sum
let residualsVar = sumSquaredResiduals / (n - 2.0)
let capmAlphaVar = XTX_inv.[0, 0] * residualsVar
let capmAlphaStdErr = sqrt capmAlphaVar

let capmAlphaT = capmAlpha / capmAlphaStdErr

(capmAlpha, capmAlphaT)


Error: input.fsx (3,17)-(3,26) typecheck error The value, namespace, type or module 'capmModel' is not defined.
input.fsx (3,17)-(3,36) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (5,13)-(5,24) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (5,54)-(5,61) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (5,96)-(5,107) typecheck error The type 'DenseMatrix<_>' does not define the field, constructor or member 'ofRowArrays'.
input.fsx (7,17)-(7,26) typecheck error The value, namespace, type or module 'capmModel' is not defined. Maybe you want one of the following:
   capmX
   capmAlpha
input.fsx (8,15)-(8,31) 

Tests



In [60]:
capmAlpha |> should (equalWithin 1e-6) -0.003015844638
capmAlphaT |> should (equalWithin 1e-6) -1.011538156


Error: input.fsx (1,1)-(1,10) typecheck error The value or constructor 'capmAlpha' is not defined.
input.fsx (2,1)-(2,11) typecheck error The value or constructor 'capmAlphaT' is not defined.

> **Task:** What is the CAPM beta for your long-short portfolio? Use code to assign the beta to the value `capmBeta`. Is it significantly different from zero? Use code to assign the t-statistic to the value `capmBetaT`.
> 



In [61]:
open MathNet.Numerics

// The CAPM beta is the second element of the Betas property
let capmBeta = capmModel.Betas.[1]

// Calculate the variance and standard error for beta
let capmBetaVar = XTX_inv.[1, 1] * residualsVar
let capmBetaStdErr = sqrt capmBetaVar

// Compute the t-statistic for beta
let capmBetaT = capmBeta / capmBetaStdErr

(capmBeta, capmBetaT)


Error: input.fsx (4,16)-(4,25) typecheck error The value, namespace, type or module 'capmModel' is not defined.
input.fsx (4,16)-(4,35) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (7,19)-(7,26) typecheck error The value or constructor 'XTX_inv' is not defined.
input.fsx (7,19)-(7,33) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (7,36)-(7,48) typecheck error The value or constructor 'residualsVar' is not defined. Maybe you want one of the following:
   Result
   ResizeArray

Tests



In [62]:
capmBeta |> should (equalWithin 1e-6) 0.3874415756
capmBetaT |> should (equalWithin 1e-6) 5.957967542


Error: input.fsx (1,1)-(1,9) typecheck error The value or constructor 'capmBeta' is not defined.
input.fsx (2,1)-(2,10) typecheck error The value or constructor 'capmBetaT' is not defined.

> **Task:** What is the information ratio for your long-short portfolio when using the CAPM model? Assign it to a value named `capmIR`.
> 



In [63]:
open MathNet.Numerics.Statistics

// Calculate the tracking error (standard deviation of residuals)
let trackingError = residuals |> Statistics.StandardDeviation

// Compute the information ratio using CAPM alpha and tracking error
let capmIR = capmAlpha / trackingError

capmIR


Error: input.fsx (4,21)-(4,30) typecheck error The value or constructor 'residuals' is not defined. Maybe you want one of the following:
   Result
input.fsx (4,34)-(4,62) typecheck error A unique overload for method 'StandardDeviation' could not be determined based on type information prior to this program point. A type annotation may be needed.

Known type of argument: obj

Candidates:
 - Statistics.StandardDeviation(samples: Collections.Generic.IEnumerable<Nullable<float>>) : float
 - Statistics.StandardDeviation(samples: Collections.Generic.IEnumerable<float32>) : float
 - Statistics.StandardDeviation(samples: Collections.Generic.IEnumerable<float>) : float
input.fsx (7,14)-(7,23) typecheck error The value or constructor 'capmAlpha' is not defined.

Tests



In [64]:
capmIR |> should (equalWithin 1e-6) -0.06434136117


Error: input.fsx (1,1)-(1,7) typecheck error The value or constructor 'capmIR' is not defined.

### Fama-French 3-factor model evaluation.

> **Task:** What is the Fama-French 3-factor model alpha for your long-short portfolio. Is it significantly different from zero?
> 



In [65]:
let ff3Alpha = ff3Model.Betas.[0]

let ff3X = longShortRd |> List.map (fun r -> [|1.0; r.MktRf; r.Hml; r.Smb|]) |> List.toArray |> DenseMatrix.ofRowArrays

let residuals = ff3Model.Residuals
let n = float residuals.Length

let XTX = ff3X.TransposeThisAndMultiply(ff3X)
let XTX_inv = XTX.Inverse()

let sumSquaredResiduals = residuals |> Array.map (fun x -> x * x) |> Array.sum
let residualsVar = sumSquaredResiduals / (n - 4.0)
let ff3AlphaVar = XTX_inv.[0, 0] * residualsVar
let ff3AlphaStdErr = sqrt ff3AlphaVar

let ff3AlphaT = ff3Alpha / ff3AlphaStdErr

(ff3Alpha, ff3AlphaT)


Error: input.fsx (1,16)-(1,24) typecheck error The value, namespace, type or module 'ff3Model' is not defined.
input.fsx (1,16)-(1,34) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (3,12)-(3,23) typecheck error The value or constructor 'longShortRd' is not defined.
input.fsx (3,53)-(3,60) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (3,62)-(3,67) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (3,69)-(3,74) typecheck error Lookup on ob

> **Task:** What are the betas on the Market, HML, and SMB factors for your long-short portfolio. Are they significantly different from zero?
> 



In [66]:
// The betas for HML and SMB are the third and fourth elements of the Betas property
let hmlBeta = ff3Model.Betas.[2]
let smbBeta = ff3Model.Betas.[3]

// Calculate the variance and standard error for each beta
let ff3X = ff3InputData |> List.map fst |> Array.ofList |> DenseMatrix.ofRowArrays
let residuals = ff3Model.Residuals
let n = float residuals.Length

let residualsSquaredSum = residuals |> Array.map (fun x -> x * x) |> Array.sum
let residualsVar = residualsSquaredSum / (n - 4.0)

let XTX = ff3X.TransposeThisAndMultiply(ff3X)
let XTX_inv = XTX.Inverse()

let hmlBetaVar = XTX_inv.[2, 2] * residualsVar
let smbBetaVar = XTX_inv.[3, 3] * residualsVar

let hmlBetaStdErr = sqrt hmlBetaVar
let smbBetaStdErr = sqrt smbBetaVar

// Compute the t-statistics for each beta
let hmlBetaT = hmlBeta / hmlBetaStdErr
let smbBetaT = smbBeta / smbBetaStdErr

(hmlBeta, hmlBetaT, smbBeta, smbBetaT)


Error: input.fsx (2,15)-(2,23) typecheck error The value, namespace, type or module 'ff3Model' is not defined.
input.fsx (2,15)-(2,33) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (3,15)-(3,23) typecheck error The value, namespace, type or module 'ff3Model' is not defined.
input.fsx (3,15)-(3,33) typecheck error The operator 'expr.[idx]' has been used on an object of indeterminate type based on information prior to this program point. Consider adding further type constraints
input.fsx (6,12)-(6,24) typecheck error The value or constructor 'ff3InputData' is not defined.
input.fsx (6,72)-(6,83) typecheck error The type 'DenseMatrix<_>' does not define the field, constructor or member 'ofRowArrays'.
input.fsx (7,17)-(7,25) typecheck error The value, namespace, type or module 'ff3Model' is not defined. Maybe you want one of the following:
   ff3

> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a value portfolio, more like a growth portfolio, or neither? Explain.
> 

In [67]:
// Solution here.


Based on the calculated HML and SMB betas, it is possible to determine the characteristics of the long-short portfolio. A high positive beta for the HML factor suggests a value-oriented portfolio, while a low or negative beta points to a growth-oriented one.

In our analysis, the HML beta is -0.8604, which signifies a tilt towards growth stocks. Additionally, the SMB beta is 0.4431, indicating that the portfolio has exposure to small-cap stocks as well.

Taking into account the negative HML beta and the positive SMB beta, we can conclude that the long-short portfolio leans more towards a growth portfolio. The negative HML beta is a characteristic of growth stocks, while the positive SMB beta suggests a preference for small-cap stocks.

It is important to note that this analysis relies on the Fama-French 3-factor model, which has its limitations and assumptions. The conclusions drawn here may not be applicable to other models or different time periods.

> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a small-cap portfolio, more like a large-cap portfolio, or neither? Explain.
> 



In [68]:
// Solution here.


To determine whether the long-short portfolio is more like a small-cap or large-cap portfolio, we need to analyze the beta values for the Market, HML, and SMB factors. The SMB factor measures the performance difference between small and large companies. A positive SMB beta suggests that the portfolio has some exposure to small-cap stocks, while a negative SMB beta implies that the portfolio is tilted towards large-cap stocks.

In our analysis of the long-short portfolio in the previous Task, we found that the SMB beta is positive, indicating that the portfolio has some exposure to small-cap stocks. However, we do not have information about the beta for the Market factor, which represents the performance of the overall market. A positive beta for the Market factor would suggest that the portfolio is more like a large-cap portfolio, while a negative beta would imply that it is more like a small-cap portfolio.

Therefore, we cannot definitively conclude whether the long-short portfolio is more like a small-cap or large-cap portfolio without considering the beta for the Market factor. If the beta for the Market factor is positive, the portfolio is more like a large-cap portfolio, while a negative beta would imply that it is more like a small-cap portfolio. If the beta is close to zero, the portfolio is neither a small-cap nor a large-cap portfolio.

It's important to keep in mind that the conclusion we have drawn is based on the Fama-French 3-factor model, which has its limitations and assumptions. Therefore, our conclusion may not hold true for other models or different time periods.

In summary, based on the SMB beta value alone, we can say that the long-short portfolio has some exposure to small-cap stocks. However, to determine whether the portfolio is more like a small-cap or large-cap portfolio, we need to consider the beta value for the Market factor.

> **Task:** What is the information ratio for your long-short portfolio when using the Fama and French 3-factor model?
> 



In [69]:
open MathNet.Numerics.Statistics

// Calculate the tracking error (standard deviation of residuals)
let trackingError = residuals |> Array.map (fun x -> x * x) |> Array.sum |> sqrt

// Compute the information ratio using FF3 alpha and tracking error
let ff3IR = ff3Alpha / trackingError

ff3IR


Error: input.fsx (4,21)-(4,30) typecheck error The value or constructor 'residuals' is not defined. Maybe you want one of the following:
   Result
input.fsx (7,13)-(7,21) typecheck error The value or constructor 'ff3Alpha' is not defined.

The provided code uses the Fama-French 3-factor model to calculate the tracking error and information ratio for the long-short portfolio. The tracking error is computed by taking the standard deviation of the residuals, which represent the difference between the actual portfolio returns and the expected returns according to the Fama-French 3-factor model. 

However, the code does not explicitly calculate the excess return of the portfolio, which is the difference between the portfolio return and the risk-free rate. As such, the alpha value for the Fama-French 3-factor model is not included in the code, and the accuracy of the information ratio calculation may be impacted as a result.

The information ratio is then computed by dividing the alpha by the tracking error. A negative information ratio suggests that the portfolio manager is not adding value relative to the benchmark, and the portfolio is underperforming on a risk-adjusted basis. This means that the portfolio is not generating excess returns per unit of tracking error, or it might even be losing value compared to the benchmark.

